In [ ]:
from bspump.jupyter import *
from bspump.http.web.server import *
import requests
import pandas as pd
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import matplotlib
import time
import io
from datetime import date
import base64
import json


In [ ]:
event = {
    "form": {
        "Year": 2024,
        "Month": 12,
        "Day": 15
    }
}


In [ ]:
today = datetime.now()

auto_pipeline(
    source=lambda app, pipeline: WebFormSource(app, pipeline, route="/",
    form_intro="""<h1>🌡️ Weather Checker for Brno</h1>
    <p>Select a date to see the weather information for Brno on that day:</p>""",
    fields=[
        IntField("Day", default=today.day, min_value=1, max_value=31),
        IntField("Month", default=today.month, min_value=1, max_value=12),
        IntField("Year", default=today.year, min_value=2020, max_value=2030),
    ]),
    sink=lambda app, pipeline: WebSink(app, pipeline)
)


In [ ]:
def output_layout(parsed_date, weather_data, img_data) -> str:
    temp = weather_data.get('temperature', 'N/A')
    feels_like = weather_data.get('feels_like', 'N/A')
    humidity = weather_data.get('humidity', 'N/A')
    description = weather_data.get('description', 'N/A')
    wind_speed = weather_data.get('wind_speed', 'N/A')
    pressure = weather_data.get('pressure', 'N/A')
    
    html = f"""
    <!DOCTYPE html>
    <html>
    <head>
        <title>Weather in Brno</title>
        <style>
            body {{
                font-family: 'Segoe UI', Tahoma, Geneva, Verdana, sans-serif;
                text-align: center;
                background: linear-gradient(135deg, #74b9ff, #0984e3);
                padding: 40px;
                margin: 0;
                min-height: 100vh;
            }}
            .container {{
                background: rgba(255, 255, 255, 0.95);
                border-radius: 20px;
                padding: 30px;
                box-shadow: 0 15px 35px rgba(0,0,0,0.1);
                max-width: 800px;
                margin: 0 auto;
            }}
            h1 {{
                color: #2d3436;
                margin-bottom: 30px;
                font-size: 2.5em;
            }}
            .weather-info {{
                display: grid;
                grid-template-columns: repeat(auto-fit, minmax(200px, 1fr));
                gap: 20px;
                margin: 30px 0;
            }}
            .weather-card {{
                background: linear-gradient(45deg, #e17055, #fdcb6e);
                border-radius: 15px;
                padding: 20px;
                color: white;
                box-shadow: 0 5px 15px rgba(0,0,0,0.1);
            }}
            .weather-card h3 {{
                margin: 0 0 10px 0;
                font-size: 1.1em;
            }}
            .weather-card .value {{
                font-size: 1.8em;
                font-weight: bold;
                margin: 0;
            }}
            img {{
                border-radius: 15px;
                max-width: 90%;
                height: auto;
                box-shadow: 0 10px 25px rgba(0,0,0,0.1);
                margin: 20px 0;
            }}
            button {{
                background: linear-gradient(45deg, #00b894, #00cec9);
                color: white;
                border: none;
                padding: 15px 30px;
                font-size: 16px;
                border-radius: 25px;
                cursor: pointer;
                transition: transform 0.2s ease;
                margin-top: 20px;
            }}
            button:hover {{
                transform: translateY(-2px);
                box-shadow: 0 5px 15px rgba(0,0,0,0.2);
            }}
        </style>
    </head>
    <body>
        <div class="container">
            <h1>🌡️ Weather in Brno on {parsed_date}</h1>
            
            <div class="weather-info">
                <div class="weather-card">
                    <h3>🌡️ Temperature</h3>
                    <p class="value">{temp}°C</p>
                </div>
                <div class="weather-card">
                    <h3>🤔 Feels Like</h3>
                    <p class="value">{feels_like}°C</p>
                </div>
                <div class="weather-card">
                    <h3>💧 Humidity</h3>
                    <p class="value">{humidity}%</p>
                </div>
                <div class="weather-card">
                    <h3>💨 Wind Speed</h3>
                    <p class="value">{wind_speed} m/s</p>
                </div>
                <div class="weather-card">
                    <h3>🔽 Pressure</h3>
                    <p class="value">{pressure} hPa</p>
                </div>
                <div class="weather-card">
                    <h3>☁️ Conditions</h3>
                    <p class="value">{description.title()}</p>
                </div>
            </div>
            
            <img src="data:image/png;base64,{base64.b64encode(img_data.getvalue()).decode()}" alt="Weather Chart">
            <br>
            <button onclick="window.history.back()">🔙 Check Another Date</button>
        </div>
    </body>
    </html>
    """
    return html


def output_no_data_found() -> str:
    html = """
    <!DOCTYPE html>
    <html>
    <head>
       <title>No Weather Data Found</title>
        <style>
            body {
                font-family: 'Segoe UI', Tahoma, Geneva, Verdana, sans-serif;
                text-align: center;
                background: linear-gradient(135deg, #636e72, #2d3436);
                padding: 40px;
                margin: 0;
                min-height: 100vh;
            }
            .container {
                background: rgba(255, 255, 255, 0.95);
                border-radius: 20px;
                padding: 40px;
                box-shadow: 0 15px 35px rgba(0,0,0,0.1);
                max-width: 600px;
                margin: 0 auto;
            }
            h1 {
                color: #d63031;
                margin-bottom: 20px;
                font-size: 2.5em;
            }
            p {
                color: #555;
                font-size: 18px;
                margin-bottom: 30px;
            }
            button {
                background: linear-gradient(45deg, #0984e3, #74b9ff);
                color: white;
                border: none;
                padding: 15px 30px;
                font-size: 16px;
                border-radius: 25px;
                cursor: pointer;
                transition: transform 0.2s ease;
            }
            button:hover {
                transform: translateY(-2px);
                box-shadow: 0 5px 15px rgba(0,0,0,0.2);
            }
        </style>
    </head>
    <body>
        <div class="container">
            <h1>⚠️ No Weather Data Found</h1>
            <p>Sorry, we couldn't retrieve weather data for the selected date. Please try a different date or check your internet connection.</p>
            <button onclick="window.history.back()">🔙 Go Back</button>
        </div>
    </body>
    </html>
    """
    return html


In [ ]:
# Global variables
BRNO_LAT = 49.1951
BRNO_LON = 16.6068

# Using Open-Meteo API (free, no API key required)
WEATHER_API_URL = "https://api.open-meteo.com/v1/forecast"
HISTORICAL_API_URL = "https://archive-api.open-meteo.com/v1/archive"

# Input processing
form = event["form"]
parsed_date_str = date(form["Year"], form["Month"], form["Day"]).strftime("%Y-%m-%d")
parsed_date = datetime.strptime(parsed_date_str, "%Y-%m-%d")
today_date = datetime.now().date()
request_date = parsed_date.date()


In [ ]:
weather_data = {}
hourly_data = []

try:
    # Determine if we need historical data or current/forecast data
    if request_date < today_date:
        # Historical data
        api_url = HISTORICAL_API_URL
    else:
        # Current or forecast data
        api_url = WEATHER_API_URL
    
    # Prepare API request
    params = {
        "latitude": BRNO_LAT,
        "longitude": BRNO_LON,
        "start_date": parsed_date_str,
        "end_date": parsed_date_str,
        "hourly": ["temperature_2m", "relative_humidity_2m", "wind_speed_10m", "surface_pressure"],
        "daily": ["temperature_2m_max", "temperature_2m_min", "apparent_temperature_max", "apparent_temperature_min"],
        "timezone": "Europe/Prague"
    }
    
    response = requests.get(api_url, params=params)
    data = response.json()
    
    if "hourly" in data and data["hourly"]:
        hourly = data["hourly"]
        daily = data.get("daily", {})
        
        # Extract hourly data for plotting
        times = hourly["time"]
        temperatures = hourly["temperature_2m"]
        humidity = hourly["relative_humidity_2m"]
        wind_speeds = hourly["wind_speed_10m"]
        pressures = hourly["surface_pressure"]
        
        # Create hourly data for plotting
        for i, time_str in enumerate(times):
            hour = datetime.fromisoformat(time_str.replace('Z', '+00:00')).hour
            hourly_data.append({
                'hour': hour,
                'temperature': temperatures[i] if temperatures[i] is not None else 0,
                'humidity': humidity[i] if humidity[i] is not None else 0,
                'wind_speed': wind_speeds[i] if wind_speeds[i] is not None else 0,
                'pressure': pressures[i] if pressures[i] is not None else 0
            })
        
        # Calculate average values for the day
        valid_temps = [t for t in temperatures if t is not None]
        valid_humidity = [h for h in humidity if h is not None]
        valid_wind = [w for w in wind_speeds if w is not None]
        valid_pressure = [p for p in pressures if p is not None]
        
        if valid_temps:
            weather_data = {
                'temperature': round(sum(valid_temps) / len(valid_temps), 1),
                'feels_like': round(sum(valid_temps) / len(valid_temps), 1),  # Simplified
                'humidity': round(sum(valid_humidity) / len(valid_humidity), 1) if valid_humidity else 0,
                'wind_speed': round(sum(valid_wind) / len(valid_wind), 1) if valid_wind else 0,
                'pressure': round(sum(valid_pressure) / len(valid_pressure), 1) if valid_pressure else 0,
                'description': f"Average conditions for {parsed_date_str}"
            }
            
            # Use daily data if available for more accurate feels like
            if daily and "apparent_temperature_max" in daily:
                if daily["apparent_temperature_max"][0] is not None:
                    weather_data['feels_like'] = round(daily["apparent_temperature_max"][0], 1)

except Exception as e:
    print(f"Error fetching weather data: {e}")
    weather_data = {}


In [ ]:
# Create visualization if data is available
if hourly_data and weather_data:
    # Create DataFrame for easier plotting
    df = pd.DataFrame(hourly_data)
    
    # Create a figure with subplots
    fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(15, 10))
    fig.suptitle(f'Weather in Brno on {parsed_date_str}', fontsize=16, fontweight='bold')
    
    # Temperature plot
    ax1.plot(df['hour'], df['temperature'], marker='o', linewidth=2, markersize=4, color='#e74c3c')
    ax1.set_title('🌡️ Temperature throughout the day', fontweight='bold')
    ax1.set_xlabel('Hour')
    ax1.set_ylabel('Temperature (°C)')
    ax1.grid(True, alpha=0.3)
    ax1.set_xticks(range(0, 24, 3))
    
    # Humidity plot
    ax2.plot(df['hour'], df['humidity'], marker='s', linewidth=2, markersize=4, color='#3498db')
    ax2.set_title('💧 Humidity throughout the day', fontweight='bold')
    ax2.set_xlabel('Hour')
    ax2.set_ylabel('Humidity (%)')
    ax2.grid(True, alpha=0.3)
    ax2.set_xticks(range(0, 24, 3))
    
    # Wind speed plot
    ax3.plot(df['hour'], df['wind_speed'], marker='^', linewidth=2, markersize=4, color='#2ecc71')
    ax3.set_title('💨 Wind Speed throughout the day', fontweight='bold')
    ax3.set_xlabel('Hour')
    ax3.set_ylabel('Wind Speed (m/s)')
    ax3.grid(True, alpha=0.3)
    ax3.set_xticks(range(0, 24, 3))
    
    # Pressure plot
    ax4.plot(df['hour'], df['pressure'], marker='D', linewidth=2, markersize=4, color='#9b59b6')
    ax4.set_title('🔽 Pressure throughout the day', fontweight='bold')
    ax4.set_xlabel('Hour')
    ax4.set_ylabel('Pressure (hPa)')
    ax4.grid(True, alpha=0.3)
    ax4.set_xticks(range(0, 24, 3))
    
    # Adjust layout
    plt.tight_layout()
    
    # Save plot to buffer
    buf = io.BytesIO()
    plt.savefig(buf, format='png', dpi=150, bbox_inches='tight')
    img_data = buf
    plt.close()


In [ ]:
if weather_data and hourly_data:
    # Build and set response with weather data
    response = output_layout(
        parsed_date=parsed_date_str,
        weather_data=weather_data,
        img_data=img_data
    )
    
    event["response"] = response
    event["content_type"] = "text/html"

else:
    # No data available
    event["response"] = output_no_data_found()
    event["content_type"] = "text/html"
